In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma  import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

In [2]:
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
import pandas as pd

books=pd.read_csv("books_cleaned.csv")

In [4]:
books


,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [5]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [6]:
books["tagged_description"].to_csv("tagged_description.csv",
                                   sep="\t",
                                   index=False,
                                   header=False
                                   )

In [7]:
raw_data=TextLoader("tagged_description.csv",encoding="utf-8").load()
text_splitter=CharacterTextSplitter(chunk_size=1  ,chunk_overlap = 0,separator="\n")
document=text_splitter.split_documents(raw_data)

Created a chunk of size 1168, which is longer than the specified 1
Created a chunk of size 1214, which is longer than the specified 1
Created a chunk of size 373, which is longer than the specified 1
Created a chunk of size 309, which is longer than the specified 1
Created a chunk of size 483, which is longer than the specified 1
Created a chunk of size 482, which is longer than the specified 1
Created a chunk of size 960, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 843, which is longer than the specified 1
Created a chunk of size 296, which is longer than the specified 1
Created a chunk of size 197, which is longer than the specified 1
Created a chunk of size 881, which is longer than the specified 1
Created a chunk of size 1088, which is longer than the specified 1
Created a chunk of size 1189, which is longer than the specified 1
Created a chunk of size 304, which is longer than the specified 1
Create

In [8]:
document[0]


Document(metadata={'source': 'tagged_description.csv'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [9]:
document[1].page_content.split(" ")[0]

'9780002261982'

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [11]:
embedder=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_books= Chroma.from_documents(document,embedding=embedder)

C:\Users\sahan\AppData\Local\Temp\ipykernel_36296\4273962042.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedder=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001A6526D6F90>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 03b9bbd9-02c6-4e06-af8c-4f51322c6020)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L

In [12]:
query = "a book to teach children about nature"
docs=db_books.similarity_search(query,k=10)
docs

[Document(id='9275c523-68dd-466c-9eda-654aad344ef8', metadata={'source': 'tagged_description.csv'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='b4a93d7d-4556-48d5-8032-bad81db9fd79', metadata={'source': 'tagged_description.csv'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='3f245ab1-ec71-4ae6-a7cc-afff5555ccc0', metadata={'source': '

In [13]:
books.loc[books["isbn13"]==int(docs[0].page_content.split(" ")[0].strip())]


,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [14]:
books[books["isbn13"]==int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [15]:
int(docs[0].page_content.split(" ")[0].strip())

9780786808069

In [16]:
def semantic_search_similarity(
        query:str,
        top_k : int = 10,

) -> pd.DataFrame:
    recs=db_books.similarity_search(query,k=50)

    books_list=[]
    for rec in recs:
        books_list += [int(rec.page_content.strip('"').split()[0])]

    return books.loc[books["isbn13"].isin(books_list)].head(top_k)

In [17]:
semantic_search_similarity("books")

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description
174,9780060593247,0060593245,Nicholas A. Basbanes,Literary Criticism,http://books.google.com/books/content?id=o4K_K...,Inspired by a landmark exhibition mounted by t...,2006.0,3.96,400.0,300.0,0,Every Book Its Reader: The Power of the Printe...,9780060593247 Inspired by a landmark exhibitio...
215,9780060760441,0060760443,Elizabeth Noble,Fiction,http://books.google.com/books/content?id=IagWj...,The Reading Group follows the trials and tribu...,2005.0,3.34,429.0,6408.0,0,The Reading Group: A Novel,9780060760441 The Reading Group follows the tr...
254,9780060841867,0060841869,Milan Kundera,Literary Collections,http://books.google.com/books/content?id=JCDXP...,Traces the author's personal view of the histo...,2006.0,3.96,176.0,1351.0,0,The Curtain: An Essay in Seven Parts,9780060841867 Traces the author's personal vie...
404,9780064402453,0064402452,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
407,9780064404419,0064404412,Laurence Yep,Juvenile Fiction,http://books.google.com/books/content?id=5AHwq...,"""Culled from 69 stories collected in a [1930s]...",1992.0,3.75,208.0,202.0,0,The Rainbow People,"9780064404419 ""Culled from 69 stories collecte..."
552,9780132197380,0132197383,Stephen Minot,Language Arts & Disciplines,http://books.google.com/books/content?id=WPjQP...,Three Genres gives students a basic introducti...,2006.0,3.65,476.0,36.0,0,Three Genres: The Writing of Fiction/Literary ...,9780132197380 Three Genres gives students a ba...
845,9780143036241,0143036246,Edward Gibbon,History,http://books.google.com/books/content?id=DAeWA...,Penguin brings you the works of the great thin...,2005.0,3.57,96.0,202.0,0,The Christians and the Fall of Rome,9780143036241 Penguin brings you the works of ...
846,9780143036357,0143036351,George Orwell,Language Arts & Disciplines,http://books.google.com/books/content?id=6I_qP...,"Throughout history, some books have changed th...",2005.0,4.03,120.0,5874.0,0,Why I Write,"9780143036357 Throughout history, some books h..."
859,9780143037576,0143037579,Søren Kierkegaard,Philosophy,http://books.google.com/books/content?id=acFp6...,"The perfect books for the true book lover, Pen...",2005.0,4.00,160.0,13333.0,0,Fear and Trembling,9780143037576 The perfect books for the true b...
874,9780143039938,0143039938,Jorge Luis Borges;Margarita Guerrero;Andrew Hu...,Fiction,http://books.google.com/books/content?id=FuNQP...,A whimsical compendium of mythological creatur...,2006.0,4.09,236.0,4809.0,0,The Book of Imaginary Beings,9780143039938 A whimsical compendium of mythol...


In [18]:
books["categories"].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Aged women,1
475,Imperialism,1
476,Human-animal relationships,1
477,Amish,1


In [19]:
books["categories"].value_counts().reset_index().query("count >= 50")

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [20]:
category_mapping={
    "Fiction":"fiction",
"Juvenile Fiction":"Children's fiction",
"Biography & Autobiography":"nonfiction",
"History":"nonfiction",
"Literary Criticism":"nonfiction",
"Religion":"nonfiction",
"Philosophy":"nonfiction",
"Comics & Graphic Novels":"fiction",
"Drama":"fiction",
"Juvenile Nonfiction":"children nonfiction",
"Science":"nonfiction",
"Poetry":"fiction",
"Literary Collections":"nonfiction"

}

In [21]:
books["simple_categories"]=books["categories"].map(category_mapping)

In [22]:
books

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,fiction
1,9780002261982,0002261987,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN
2,9780006178736,0006178731,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",fiction
3,9780006280897,0006280897,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN
4,9780006280934,0006280935,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,NaN
5193,9788173031014,8173031010,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,0,Journey to the East,9788173031014 This book tells the tale of a ma...,NaN
5194,9788179921623,817992162X,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,NaN
5195,9788185300535,8185300534,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,nonfiction


In [23]:
books[~(books["simple_categories"].isna())]

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,fiction
2,9780006178736,0006178731,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",fiction
8,9780006482079,0006482074,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,fiction
30,9780006646006,000664600X,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...,Children's fiction
31,9780007105045,0007105045,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,0,Tree and Leaf: The Homecoming of Beorhtnoth : ...,"9780007105045 ""The two works 'On fairy-stories...",nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,9781933648279,1933648279,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,0,Night Has a Thousand Eyes,"9781933648279 ""Cornell Woolrich's novels defin...",fiction
5188,9784770028969,4770028962,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,fiction
5189,9788122200850,8122200850,Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,fiction
5195,9788185300535,8185300534,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,nonfiction


In [24]:
from transformers import pipeline
labels=["fiction","nonfiction","childern/'s nonfiction", "children/'s fiction"]
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)


Device set to use cpu


In [25]:
sequence=books.loc[books["simple_categories"]=="fiction","description"].reset_index(drop=True)[0]

In [26]:
sequence

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [27]:
labels=['fiction','nonfiction',"children's fiction", "children's nonfiction"]
pipe(sequence,labels)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [28]:
import numpy as np
#argmax finds the index of maximum value
max_index=np.argmax(pipe(sequence,labels)["scores"])
max_label=pipe(sequence,labels)["labels"][max_index]

In [29]:
max_label
labels=['fiction','nonfiction']



In [30]:
def generate_prediction(
        sequence :str ,
        categories : list
      )-> str:
    max_index=np.argmax(pipe(sequence,categories)["scores"])
    max_label=pipe(sequence,categories)["labels"][max_index]
    return max_label


In [31]:
sequence1=books.loc[books["simple_categories"]=="nonfiction","description"].reset_index(drop=True)[0]


In [32]:
ans=generate_prediction(sequence1,labels)
ans

'nonfiction'

In [47]:
from tqdm import tqdm
# long loops looks like they are stuck therefore tqdm is used which shows the percentage
#     speed,items processed etc

In [48]:
actual_cats=[]
pred_cats=[]

for i in tqdm(range(0,300)):
    sequence=books.loc[books["simple_categories"]=="fiction","description"].reset_index(drop=True)[i]
    actual_cats += ["fiction"]
    pred_cats +=[generate_prediction(sequence,labels)]

100%|██████████| 300/300 [08:54<00:00,  1.78s/it]


In [49]:
for i in tqdm(range(0,300)):
    sequence=books.loc[books["simple_categories"]=="nonfiction","description"].reset_index(drop=True)[i]
    actual_cats += ["nonfiction"]
    pred_cats +=[generate_prediction(sequence,labels)]


100%|██████████| 300/300 [09:45<00:00,  1.95s/it]


In [36]:
# for i in tqdm(range(0,300)):
#     sequence=books.loc[books["simple_categories"]=="children's fiction","description"].reset_index(drop=True)[i]
#     actual_cats += ["children's fiction"]
#     pred_cats +=generate_prediction(sequence,labels)


In [37]:
# for i in tqdm(range(0,300)):
#     sequence=books.loc[books["simple_categories"]=="children's nonfiction","description"].reset_index(drop=True)[i]
#     actual_cats += ["children's nonfiction"]
#     pred_cats +=generate_prediction(sequence,labels)


In [52]:
predicted_categories = pd.DataFrame({"actuals_categories": actual_cats, "predicted_categories": pred_cats})


In [50]:
len(actual_cats)

600

In [51]:
len(pred_cats)

600

In [53]:
pred_cats

['fiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'fiction',
 'fiction',
 'fiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'fiction',
 'fiction',
 'fiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'fiction',
 'fiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'fiction',
 'fiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'fiction',
 'fiction',
 'nonfiction',
 'fiction',
 'fiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'fiction',
 'fiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'fiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'nonfiction',
 'fiction',
 'nonfiction',
 'no

In [55]:
predicted_categories


,actuals_categories,predicted_categories
0,fiction,fiction
1,fiction,nonfiction
2,fiction,nonfiction
3,fiction,nonfiction
4,fiction,fiction
...,...,...
595,nonfiction,nonfiction
596,nonfiction,nonfiction
597,nonfiction,nonfiction
598,nonfiction,fiction


In [58]:
predicted_categories["correct_prediction"]=(np.where(predicted_categories["actuals_categories"]==predicted_categories["predicted_categories"],1,0))

In [59]:
predicted_categories["correct_prediction"].sum()/len(predicted_categories)

np.float64(0.6816666666666666)

In [61]:
books.loc[books["simple_categories"].isna()]

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description,simple_categories
1,9780002261982,0002261987,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN
3,9780006280897,0006280897,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN
4,9780006280934,0006280935,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN
5,9780006380832,0006380832,Richard Hall,"Africa, East",http://books.google.com/books/content?id=MuPEQ...,Until Vasco da Gama discovered the sea-route t...,1998.0,4.41,608.0,65.0,0,Empires of the Monsoon: A History of the India...,9780006380832 Until Vasco da Gama discovered t...,NaN
6,9780006470229,000647022X,Stephen R. Donaldson,"Hyland, Morn (Fictitious character)",http://books.google.com/books/content?id=4oXav...,A new-cover reissue of the fourth book in the ...,1994.0,4.15,743.0,103.0,0,The Gap Into Madness: Chaos and Order,9780006470229 A new-cover reissue of the fourt...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5190,9788125026600,8125026606,Marc Sautet,NaN,http://books.google.com/books/content?id=fPIv8...,Not only does Nietzsche for Beginners delve in...,2004.0,3.26,192.0,21.0,0,Nietzsche For Beginners,9788125026600 Not only does Nietzsche for Begi...,NaN
5191,9788171565641,8171565646,E. M. Forster,English fiction,http://books.google.com/books/content?id=qWU9P...,"Forster's lively, informed originality and wit...",2004.0,3.83,141.0,10.0,0,Aspects of the Novel,"9788171565641 Forster's lively, informed origi...",NaN
5192,9788172235222,8172235224,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,NaN
5193,9788173031014,8173031010,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,0,Journey to the East,9788173031014 This book tells the tale of a ma...,NaN


In [63]:
isbn=[]
pred_cats=[]

missing_cats_books=books.loc[books["simple_categories"].isna(),["isbn13","description"]].reset_index(drop=True)

In [64]:

for i in tqdm(range(0,len(missing_cats_books))):
    sequence=missing_cats_books["description"][i]
    pred_cats += [generate_prediction(sequence,labels)]
    isbn += [missing_cats_books["isbn13"][i]]

100%|██████████| 1404/1404 [43:08<00:00,  1.84s/it] 


In [65]:
missing_pred_df=pd.DataFrame({"isbn13": isbn, "predicted_categories": pred_cats})

In [66]:
missing_pred_df

,isbn13,predicted_categories
0,9780002261982,fiction
1,9780006280897,nonfiction
2,9780006280934,nonfiction
3,9780006380832,nonfiction
4,9780006470229,fiction
...,...,...
1399,9788125026600,nonfiction
1400,9788171565641,fiction
1401,9788172235222,nonfiction
1402,9788173031014,nonfiction


In [67]:
books=pd.merge(books,missing_pred_df,on="isbn13",how="left")
books["simple_categories"]=np.where(books["simple_categories"].isna(),books["predicted_categories"],books["simple_categories"])

In [68]:
books.loc[books["simple_categories"].isna()]

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description,simple_categories,predicted_categories


In [69]:
books=books.drop(columns=["predicted_categories"])

In [70]:
books

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,fiction
1,9780002261982,0002261987,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,fiction
2,9780006178736,0006178731,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",fiction
3,9780006280897,0006280897,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,nonfiction
4,9780006280934,0006280935,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,nonfiction
5193,9788173031014,8173031010,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,0,Journey to the East,9788173031014 This book tells the tale of a ma...,nonfiction
5194,9788179921623,817992162X,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,fiction
5195,9788185300535,8185300534,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,nonfiction


In [72]:
books[books["categories"].str.lower().isin([
    "romance",
    "science fiction",
    "scifi",
    "fantasy",
    "mystery",
    "thriller",
    "comedy",
    "crime",
    "historical"
])]

,isbn13,isbn10,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description,simple_categories
24,9780006513087,0006513085,Tess Gerritsen,Science fiction,http://books.google.com/books/content?id=KI66c...,Emma Watson a research physician has been trai...,2004.0,4.04,342.0,8024.0,0,Gravity,9780006513087 Emma Watson a research physician...,nonfiction
475,9780099410355,0099410354,Matthew Woodring Stover,Science fiction,http://books.google.com/books/content?id=VbICO...,"From the depths of catastrophe, a glimmer of h...",2002.0,4.00,320.0,6765.0,0,Traitor,"9780099410355 From the depths of catastrophe, ...",nonfiction
478,9780099422341,0099422344,Joseph O'Connor,Comedy,http://books.google.com/books/content?id=DrE3I...,"In aid of Amnesty International, this is a bri...",2002.0,3.39,298.0,34.0,0,Yeats is Dead!: A Novel by Fifteen Irish Writers,"9780099422341 In aid of Amnesty International,...",fiction
1090,9780261102422,0261102427,John Ronald Reuel Tolkien,Fantasy,http://books.google.com/books/content?id=22ePu...,Tolkien's Silmarillion is the core work of the...,1999.0,3.91,384.0,253.0,0,The Silmarillion,9780261102422 Tolkien's Silmarillion is the co...,fiction
1435,9780340837955,0340837950,Robert A. Heinlein,Science fiction,http://books.google.com/books/content?id=ZQhiP...,"Epic, entertaining, Stranger in a Strange Land...",2005.0,3.92,672.0,563.0,0,Stranger in a Strange Land,"9780340837955 Epic, entertaining, Stranger in ...",fiction
1439,9780345251220,0345251229,William Arrow,Science fiction,NaN,"The first novel in the series, ""Return to the ...",1976.0,3.23,183.0,10.0,0,Visions from Nowhere,"9780345251220 The first novel in the series, ""...",fiction
2845,9780575075597,0575075597,Ken Grimwood,Fantasy,http://books.google.com/books/content?id=9vmNP...,At forty-three Jeff Winston is tired of his lo...,2005.0,4.16,272.0,412.0,0,Replay,9780575075597 At forty-three Jeff Winston is t...,nonfiction
2860,9780590254762,0590254766,Clive Staples Lewis,Fantasy,NaN,Four English school children enter the magic l...,1995.0,4.21,189.0,860.0,0,"The lion, the witch and the wardrobe",9780590254762 Four English school children ent...,nonfiction
3288,9780739423851,0739423851,Diana Wynne Jones,Fantasy,http://books.google.com/books/content?id=hB7hA...,Howl's moving castle - Eldest of three sisters...,2002.0,4.44,376.0,439.0,0,Wizard's Castle,9780739423851 Howl's moving castle - Eldest of...,nonfiction
3289,9780739439708,0739439707,Madeleine L'Engle,Science fiction,NaN,"Blending magic with quantum physics, Madeleine...",2003.0,4.35,646.0,165.0,0,Time Quartet,9780739439708 Blending magic with quantum phys...,fiction


In [73]:
books.to_csv("book_with_categories.csv",index=False)

In [74]:
#sentiment analysis